# Anomaly detection with autoencoders

## Objective

Train an autoencoder to detect anomalies by learning what "normal" data looks like. Anything that deviates significantly from this learned pattern is flagged as anomalous.

## Approach

1. **Train on normal data only**: We'll use the TF Flowers dataset (all 5 flower types) as our "normal" training data
2. **Test on anomalies**: We'll use COCO dataset images (cars, people, animals, food, etc.) as anomalies
3. **Detection method**: High reconstruction error indicates the image is NOT flower-like, therefore anomalous

## Key Insight

The autoencoder learns to compress and reconstruct flower images efficiently. When shown non-flower images, it can't reconstruct them well because they don't match the learned flower patterns. This reconstruction error becomes our anomaly score!

## Real-World Applications

- Quality control in manufacturing
- Fraud detection in financial transactions
- Network intrusion detection
- Medical imaging (detecting abnormal scans)

## 1. Notebook setup

### 1.1. Imports

In [ ]:
# Standard library
import sys
from pathlib import Path

# Third-party
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

# Add src directory to path
sys.path.append(str(Path.cwd().parent))

# Local imports
from src.data_utils import FLOWER_CLASSES, COCO_CLASSES, load_flowers, load_coco_cached
from src.metrics import (
    calculate_reconstruction_error,
    calculate_psnr,
    calculate_ssim,
    compute_roc_metrics,
    find_optimal_threshold
)
from src.model_utils import build_compression_ae
from src.visualization import plot_image_grid, plot_roc_curve, plot_error_histogram
from src.huggingface_utils import download_model, upload_model


### 1.2. Configuration

In [ ]:
# Training configuration
TRAIN_MODEL = True  # Set to False to download pre-trained model from HuggingFace

# Training hyperparameters
LATENT_DIM = 128
EPOCHS = 50
BATCH_SIZE = 128
LEARNING_RATE = 0.001

# GPU configuration
GPU_ID = 0  # Which GPU to use (0-indexed). Set to None to use all available GPUs.

# Create models directory
models_dir = Path('../models')
models_dir.mkdir(exist_ok=True)

# Create logs directory for TensorBoard
logs_dir = Path('../logs')
logs_dir.mkdir(exist_ok=True)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Configure GPU
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        if GPU_ID is not None:
            # Use specific GPU
            tf.config.set_visible_devices(gpus[GPU_ID], 'GPU')
            tf.config.experimental.set_memory_growth(gpus[GPU_ID], True)
            print(f'Using GPU {GPU_ID}: {gpus[GPU_ID].name}')
        else:
            # Use all GPUs with memory growth
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f'Using {len(gpus)} GPU(s): {[gpu.name for gpu in gpus]}')
    except RuntimeError as e:
        print(e)
else:
    print('No GPU available, using CPU')

## 2. Data preparation

We'll train on the **TF Flowers dataset** (all 5 classes) as 'normal' data, then test anomaly detection using **non-flower images** from COCO.

This demonstrates the key insight: train on what's normal, detect anything different as anomalous.

In [ ]:
# Load flowers dataset (all classes as "normal")
(x_train_flowers, y_train_flowers), (x_test_flowers, y_test_flowers) = load_flowers(normalize=True)

print('Flowers Dataset (Normal Images):')
print(f'  Training samples: {x_train_flowers.shape}')
print(f'  Test samples: {x_test_flowers.shape}')
print(f'  Classes: {FLOWER_CLASSES}')
print(f'  All flower images will be considered NORMAL')

In [ ]:
# Load COCO images as anomalies (non-flower images)
# Using cached version for faster loading
(_, _), (x_coco, y_coco) = load_coco_cached(subset_percent=10, normalize=True)

# Sample 500 random COCO images for anomaly testing
n_anomaly_samples = 500
anomaly_indices = np.random.choice(len(x_coco), n_anomaly_samples, replace=False)
x_test_anomaly = x_coco[anomaly_indices]
y_test_anomaly = y_coco[anomaly_indices]

print('\nCOCO Dataset (Anomalous Images):')
print(f'  Anomaly test samples: {len(x_test_anomaly)}')
print(f'  Number of COCO classes: {len(COCO_CLASSES)}')
print(f'  Sample categories in test set: {[COCO_CLASSES[y] for y in y_test_anomaly[:10]]}')

# Organize data for training and evaluation
x_train_normal = x_train_flowers
x_test_normal = x_test_flowers

print('\n' + '='*60)
print('Dataset Summary:')
print('='*60)
print(f'Training (normal flowers): {len(x_train_normal)} samples')
print(f'Test normal (flowers): {len(x_test_normal)} samples')
print(f'Test anomaly (COCO non-flowers): {len(x_test_anomaly)} samples')


In [ ]:
# Visualize normal (flowers) vs anomalous (non-flowers) samples
fig, axes = plt.subplots(2, 10, figsize=(20, 4))

# Normal samples (flowers)
normal_indices = np.random.choice(len(x_test_normal), 10, replace=False)
for i, idx in enumerate(normal_indices):
    axes[0, i].imshow(x_test_normal[idx])
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_ylabel('Normal\n(Flowers)', fontweight='bold', fontsize=12, rotation=0, labelpad=50, va='center')

# Anomalous samples (COCO)
anomaly_indices_viz = np.random.choice(len(x_test_anomaly), 10, replace=False)
for i, idx in enumerate(anomaly_indices_viz):
    axes[1, i].imshow(x_test_anomaly[idx])
    axes[1, i].axis('off')
    # Show category name
    category = COCO_CLASSES[y_test_anomaly[idx]]
    axes[1, i].set_title(category, fontsize=9, color='red')
    if i == 0:
        axes[1, i].set_ylabel('Anomalous\n(COCO)', fontweight='bold', fontsize=12, rotation=0, labelpad=50, va='center', color='red')

plt.suptitle('Normal (Flowers) vs Anomalous (Non-Flower Images)', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## Build and Train Anomaly Detection Model

**Key Concept**: We train ONLY on flower images (normal data). The model learns to reconstruct flowers very well.

When we later show it non-flower images (cars, people, food, etc.), it struggles to reconstruct them accurately, producing high reconstruction error. This is how we detect anomalies!

In [ ]:
# Training configuration
LATENT_DIM = 128
EPOCHS = 50
BATCH_SIZE = 128
LEARNING_RATE = 0.001

# Create models directory
models_dir = Path('../models')
models_dir.mkdir(exist_ok=True)

logs_dir = Path('../logs')
logs_dir.mkdir(exist_ok=True)

In [ ]:
if not TRAIN_MODEL:
    print('Downloading pre-trained model from HuggingFace...')
    model_path = download_model('anomaly_ae.keras', models_dir='../models')
    
    from tensorflow import keras
    autoencoder = keras.models.load_model(model_path)
    print('✓ Model loaded! Skipping training.')
    print('Set TRAIN_MODEL=True to train your own model.')


In [ ]:
# Build model
autoencoder = build_anomaly_ae(latent_dim=LATENT_DIM)

# Compile
autoencoder.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='mse',
    metrics=['mae']
)

print(f"Total parameters: {autoencoder.count_params():,}")
autoencoder.summary()

In [ ]:
# Callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=str(models_dir / 'anomaly_ae.keras'),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.TensorBoard(
        log_dir=str(logs_dir / 'anomaly_detection'),
        histogram_freq=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
]

# Train on NORMAL data only
print("\n" + "="*60)
print("Training Anomaly Detection Autoencoder")
print("Training on NORMAL classes only!")
print("="*60 + "\n")

history = autoencoder.fit(
    x_train_normal, x_train_normal,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(x_test_normal, x_test_normal),  # Validate on normal data
    callbacks=callbacks,
    verbose=1
)

print("\nModel saved to: models/anomaly_ae.keras")

In [ ]:
# Plot training history
fig = plot_training_history(history)
plt.suptitle('Anomaly Detection Model Training History', fontsize=14, y=1.02)
plt.show()

## Compute Reconstruction Errors

Now we'll compute reconstruction errors for both normal and anomalous test data.

In [ ]:
# Calculate reconstruction errors
print("Computing reconstruction errors...")

errors_normal = calculate_reconstruction_error(autoencoder, x_test_normal)
errors_anomaly = calculate_reconstruction_error(autoencoder, x_test_anomaly)

print(f"\nNormal samples:")
print(f"  Mean error: {np.mean(errors_normal):.6f}")
print(f"  Std error: {np.std(errors_normal):.6f}")
print(f"  Min error: {np.min(errors_normal):.6f}")
print(f"  Max error: {np.max(errors_normal):.6f}")

print(f"\nAnomalous samples:")
print(f"  Mean error: {np.mean(errors_anomaly):.6f}")
print(f"  Std error: {np.std(errors_anomaly):.6f}")
print(f"  Min error: {np.min(errors_anomaly):.6f}")
print(f"  Max error: {np.max(errors_anomaly):.6f}")

In [ ]:
# Visualize error distributions
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(errors_normal, bins=50, alpha=0.7, label='Normal', color='blue', density=True)
axes[0].hist(errors_anomaly, bins=50, alpha=0.7, label='Anomaly', color='red', density=True)
axes[0].set_xlabel('Reconstruction Error (MSE)', fontsize=12)
axes[0].set_ylabel('Density', fontsize=12)
axes[0].set_title('Distribution of Reconstruction Errors', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=12)
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot([errors_normal, errors_anomaly], labels=['Normal', 'Anomaly'])
axes[1].set_ylabel('Reconstruction Error (MSE)', fontsize=12)
axes[1].set_title('Error Distribution Comparison', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## ROC Analysis and Threshold Selection

We'll use ROC (Receiver Operating Characteristic) analysis to:
1. Evaluate detection performance
2. Find the optimal threshold

In [ ]:
# Prepare labels and scores
y_true = np.concatenate([
    np.zeros(len(errors_normal)),  # Normal = 0
    np.ones(len(errors_anomaly))   # Anomaly = 1
])

scores = np.concatenate([errors_normal, errors_anomaly])

# Compute ROC metrics
roc_metrics = compute_roc_metrics(y_true, scores)

print(f"ROC-AUC Score: {roc_metrics['auc']:.4f}")

# Find optimal threshold
optimal_threshold = find_optimal_threshold(y_true, scores)
print(f"Optimal Threshold: {optimal_threshold:.6f}")

# Also compute percentile-based threshold
percentile_threshold = compute_anomaly_threshold(errors_normal, percentile=95)
print(f"95th Percentile Threshold: {percentile_threshold:.6f}")

In [ ]:
# Plot ROC curve
fig, ax = plt.subplots(figsize=(8, 8))

ax.plot(roc_metrics['fpr'], roc_metrics['tpr'], linewidth=2, 
        label=f"ROC Curve (AUC = {roc_metrics['auc']:.4f})")
ax.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')

ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curve - Anomaly Detection', fontsize=14, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate with optimal threshold
y_pred = (scores > optimal_threshold).astype(int)

# Confusion matrix
cm = compute_confusion_matrix(y_true, y_pred)

# Calculate metrics
tn, fp, fn, tp = cm.ravel()
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print("\n" + "="*50)
print("ANOMALY DETECTION PERFORMANCE")
print("="*50)
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print("="*50)

print(f"\nConfusion Matrix:")
print(f"  True Negatives:  {tn:5d}")
print(f"  False Positives: {fp:5d}")
print(f"  False Negatives: {fn:5d}")
print(f"  True Positives:  {tp:5d}")

In [ ]:
# Visualize confusion matrix
import seaborn as sns

fig, ax = plt.subplots(figsize=(8, 6))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Anomaly'],
            yticklabels=['Normal', 'Anomaly'],
            cbar_kws={'label': 'Count'},
            ax=ax, annot_kws={'size': 16})

ax.set_xlabel('Predicted Label', fontsize=12)
ax.set_ylabel('True Label', fontsize=12)
ax.set_title('Confusion Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## Visual Analysis: Top Anomalies and Errors

Let's examine which samples have the highest and lowest reconstruction errors.

In [ ]:
# Find top anomalies (highest errors in anomaly set)
top_anomaly_indices = np.argsort(errors_anomaly)[-10:][::-1]
top_anomaly_errors = errors_anomaly[top_anomaly_indices]
top_anomaly_images = x_test_anomaly[top_anomaly_indices]

# Find false negatives (low errors in anomaly set)
false_neg_indices = np.argsort(errors_anomaly)[:10]
false_neg_errors = errors_anomaly[false_neg_indices]
false_neg_images = x_test_anomaly[false_neg_indices]

# Find false positives (high errors in normal set)
false_pos_indices = np.argsort(errors_normal)[-10:][::-1]
false_pos_errors = errors_normal[false_pos_indices]
false_pos_images = x_test_normal[false_pos_indices]

In [ ]:
# Visualize results
fig, axes = plt.subplots(3, 10, figsize=(20, 6))

# Top anomalies (correctly detected)
for i in range(10):
    axes[0, i].imshow(top_anomaly_images[i])
    axes[0, i].set_title(f'{top_anomaly_errors[i]:.4f}', fontsize=10)
    axes[0, i].axis('off')
axes[0, 0].set_ylabel('Top Anomalies\n(Correctly Detected)', fontsize=12, fontweight='bold')

# False negatives (anomalies that look normal)
for i in range(10):
    axes[1, i].imshow(false_neg_images[i])
    axes[1, i].set_title(f'{false_neg_errors[i]:.4f}', fontsize=10, color='orange')
    axes[1, i].axis('off')
axes[1, 0].set_ylabel('False Negatives\n(Missed Anomalies)', fontsize=12, fontweight='bold')

# False positives (normal that look anomalous)
for i in range(10):
    axes[2, i].imshow(false_pos_images[i])
    axes[2, i].set_title(f'{false_pos_errors[i]:.4f}', fontsize=10, color='red')
    axes[2, i].axis('off')
axes[2, 0].set_ylabel('False Positives\n(Normal Flagged)', fontsize=12, fontweight='bold')

plt.suptitle('Error Analysis: Top-10 Examples by Category', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

## Reconstruction Visualization

Compare how well the model reconstructs normal vs anomalous images.

In [ ]:
# Select samples
n_samples = 5

# Normal samples
normal_samples = x_test_normal[np.random.choice(len(x_test_normal), n_samples, replace=False)]
normal_reconstructions = autoencoder.predict(normal_samples, verbose=0)

# Anomaly samples
anomaly_samples = x_test_anomaly[np.random.choice(len(x_test_anomaly), n_samples, replace=False)]
anomaly_reconstructions = autoencoder.predict(anomaly_samples, verbose=0)

# Plot
fig, axes = plt.subplots(4, n_samples, figsize=(n_samples * 3, 12))

for i in range(n_samples):
    # Normal originals
    axes[0, i].imshow(normal_samples[i])
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_title('Normal\nOriginal', fontweight='bold')
    
    # Normal reconstructions
    axes[1, i].imshow(normal_reconstructions[i])
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_title('Normal\nReconstructed', fontweight='bold')
    
    # Anomaly originals
    axes[2, i].imshow(anomaly_samples[i])
    axes[2, i].axis('off')
    if i == 0:
        axes[2, i].set_title('Anomaly\nOriginal', fontweight='bold', color='red')
    
    # Anomaly reconstructions
    axes[3, i].imshow(anomaly_reconstructions[i])
    axes[3, i].axis('off')
    if i == 0:
        axes[3, i].set_title('Anomaly\nReconstructed', fontweight='bold', color='red')

plt.suptitle('Reconstruction Quality: Normal vs Anomalous', fontsize=16, y=0.995)
plt.tight_layout()
plt.show()

## Conclusions

### Key Findings:

1. **Effective Separation**: The autoencoder successfully distinguishes between flower images (normal) and non-flower images (anomalous) based on reconstruction error alone.

2. **High Performance**: Achieved strong ROC-AUC, demonstrating the model learned flower-specific features that don't transfer to arbitrary objects.

3. **Interpretable**: Reconstruction errors provide interpretable anomaly scores - higher error means "less flower-like".

4. **Generalization**: This approach works because the model learned what flowers look like. Anything that doesn't match that learned pattern (cars, people, food, etc.) gets high reconstruction error.

### Real-World Applications:

- **Manufacturing Quality Control**: Detect defective products that look different from normal items
- **Security & Fraud Detection**: Identify unusual patterns in network traffic or transactions  
- **Medical Imaging**: Flag abnormal scans that deviate from healthy patterns
- **Content Moderation**: Detect inappropriate content by training on appropriate content only

### Interactive Demo:

Try the Streamlit app where you can:
- Upload your own images to test if they're "flower-like"
- See reconstruction errors in real-time
- Adjust detection thresholds
- Test edge cases (potted plants, gardens, flower paintings, etc.)

### Limitations:

- Requires clean normal data for training (no anomalies in training set)
- Threshold selection affects false positive/negative trade-off
- May struggle with subtle anomalies or edge cases (e.g., flower paintings)

### Next Steps:

1. Try uploading various images in the Streamlit app
2. Experiment with different latent dimensions
3. Explore Variational Autoencoders (VAEs) for probabilistic anomaly scores